In [37]:
# Download Guteburg Book of Mormon and Bible
!curl -L https://ia601205.us.archive.org/18/items/thebookofmormon00017gut/mormon13.txt > bookOfMormon.txt
!curl -L "https://archive.org/stream/thegutembergbible/The%20Gutemberg%20Bible%201454_djvu.txt" > bible.txt

# Setup Spark SQL
%pip install pyspark
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BibleCounter").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1551k  100 1551k    0     0  2827k      0 --:--:-- --:--:-- --:--:-- 2836k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  499k    0  499k    0     0  1490k      0 --:--:-- --:--:-- --:--:-- 1491k


Note: you may need to restart the kernel to use updated packages.


In [43]:
# Get a table of words for Book of Mormon and Bible
import re
bookOfMormonLines = sc.textFile("bookOfMormon.txt")
bookOfMormonLinesLower = bookOfMormonLines.map(lambda line: line.lower())
bookOfMormonWords = bookOfMormonLinesLower.flatMap(lambda lowerLine: re.split('[^a-z]', lowerLine))

print(bookOfMormonWords.take(10))

bibleLines = sc.textFile("bookOfMormon.txt")
bibleLinesLower = bibleLines.map(lambda line: line.lower())
bibleWords = bibleLinesLower.flatMap(lambda lowerLine: re.split('[^a-z]', lowerLine))

print(bibleWords.take(10))


['', '', '', '', '', 'this', 'is', 'the', 'project', 'gutenberg']
['', '', '', '', '', 'this', 'is', 'the', 'project', 'gutenberg']


In [47]:
# Save them as tables that we can query with SQL-like syntax
from pyspark.sql.types import *
schema = StructType([
    StructField("word", StringType(), True)
])
bookOfMormonWords.toDF().show()#.createOrReplaceTempView('bookOfMormon_words')

bibleWords.toDF(schema).createOrReplaceTempView('bible_words')

In [41]:
result = spark.sql("select * from bookOfMormon_words except select * from bible_words")

result.show()

Py4JJavaError: An error occurred while calling o411.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 24) (host.docker.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
    process()
  File "C:\Users\micha\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "C:\Users\micha\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\micha\AppD